<a href="https://colab.research.google.com/github/sherna90/matematicas_discretas/blob/master/10.-algoritmo_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 4.2 MB/s 


In [96]:
public_exponent=65537

In [98]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

private_key = rsa.generate_private_key(public_exponent=public_exponent,key_size=512,backend=default_backend())
# Extract the public key from the private key.
public_key = private_key.public_key()
private_key_bytes = private_key.private_bytes(encoding=serialization.Encoding.PEM,
                                              format=serialization.PrivateFormat.TraditionalOpenSSL,encryption_algorithm=serialization.NoEncryption())
# Convert the public key into bytes.
public_key_bytes = public_key.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo)

# Convert the private key bytes back to a key.
# Because there is no encryption of the key, there is no password.
private_key = serialization.load_pem_private_key(private_key_bytes,backend=default_backend(),password=None)
public_key = serialization.load_pem_public_key(public_key_bytes,backend=default_backend())

In [108]:
public_key.public_numbers()

<RSAPublicNumbers(e=65537, n=10696687679368744404803073101319228517687924352091391894063774552535887477893597679773514178491119786400897663106462072593407825313233810287193946799810009)>

In [100]:
e,n=public_key.public_numbers().e,public_key.public_numbers().n

In [101]:
p,q=private_key.private_numbers().p,private_key.private_numbers().q

In [102]:
def gcdExtended(a, b):
    # Base Case
    if a == 0 : 
        return b, 0, 1
    gcd, x1, y1 = gcdExtended(b%a, a)
    # Update x and y using results of recursive
    # call
    x = y1 - (b//a) * x1
    y = x1    
    return gcd, x, y


In [117]:
private_key.private_numbers().d

306846099717918725010753886056428423840781509405859541340615166375749095296390961717767476989952622735540030868995184536512289394593762996849881336398433

In [118]:
gcd,x,y=gcdExtended(e,(p-1)*(q-1))

In [34]:
def powmod(a: int, e: int, n: int) -> int:
    accum: int = 1
    apow2: int = a
    while e > 0:
        if e & 1:
            accum = (accum * apow2) % n
        apow2 = (apow2 * apow2) % n
        e = e >> 1
    return accum

def simple_rsa_encrypt(m, publickey):
  numbers = publickey.public_numbers()
  return powmod(m, numbers.e, numbers.n)

def simple_rsa_decrypt(c, privatekey):
  numbers = privatekey.private_numbers()
  return powmod(c, numbers.d, numbers.public_numbers.n)

def int_to_bytes(i):
  i = int(i)
  return i.to_bytes((i.bit_length()+7)//8, byteorder='big')

def bytes_to_int(b):
  return int.from_bytes(b, byteorder='big')